In [1]:
from __future__ import division
import  nltk
import  re
import string
import random
import pandas as pd
import numpy as np
import urllib
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import nltk.classify.util
import warnings





from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk import classify
from nltk import NaiveBayesClassifier
from nltk import TreebankWordTokenizer, SnowballStemmer
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import PorterStemmer
from string import punctuation 
from nltk.corpus import sentiwordnet as swn





from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC

from sklearn.naive_bayes import BernoulliNB
from collections import Counter

train =pd.read_csv('../input/climate-change-edsa2020/train.csv')
test =pd.read_csv('../input/climate-change-edsa2020/test.csv')

stop_words = set(stopwords.words('english'))

#Assigning a sentiment

train.sentiment.value_counts()


#combining the two sets 
combination = train.append(test, ignore_index=True)

#removing noise

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt  
combination['clean_tweet'] = np.vectorize(remove_pattern)(combination['message'], "@[\w]*")
combination['clean_tweet'] = combination['clean_tweet'].str.replace("[^a-zA-Z#]", " ")
combination['clean_tweet'] = combination['clean_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# Tokenising the tweets
tokenized_tweet = combination['clean_tweet'].apply(lambda x: x.split())

for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

combination['clean_tweet'] = tokenized_tweet

#Stemming

stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) 




y = train['sentiment']
X = train['message']

vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2, stop_words="english")
X_vectorized = vectorizer.fit_transform(X)

X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size=.3,shuffle=True, stratify=y, random_state=11)

sgd = SGDClassifier(loss="hinge",max_iter=20)
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)
f1_score(y_val, sgd_pred, average="macro")

testx = test['message']
test_vect = vectorizer.transform(testx)
y_pred = sgd.predict(test_vect)
test['sentiment'] = y_pred
test[['tweetid','sentiment']].to_csv('classification2.csv', index=False)









/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
